In [1]:
import numpy as np
import pandas as pd

### <font color='green'>Loading and Cleaning</font>

In [13]:
df = pd.read_csv('US Police shootings in from 2015-22.csv')
df.tail(2)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
7727,7728,Tyler Michael Gardner,2022-09-07,shot,gun,27.0,M,NaN,Washington County,TN,False,attack,Other,False,-82.539,36.309,True
7728,7729,Reginald Lamont Grant,2022-09-07,shot,gun,30.0,M,NaN,Columbus,GA,False,attack,Foot,False,NaN,NaN,True


In [14]:
df.drop(['id'], axis=1, inplace=True)

In [15]:
# from data_utils import extract_from_date
# df = extract_from_date(df, ['date'])
def extract_from_date(data, cols):
    for col in cols:
        data[col] = pd.to_datetime(data[col])
        # for fet in ['year', 'month_name', 'day_name']:
        data['year'] = data[col].dt.year
        data['month'] = data[col].dt.month
        data['day_name'] = data['date'].dt.day_name()
    return data

In [16]:
df = extract_from_date(df, ['date'])
df.head(1)

,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact,year,month,day_name
0,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True,2015,1,Friday


In [6]:
for col in df.columns:
    if df[col].dtype not in ['float64', '<M8[ns]', 'bool']:
        df[col] = df[col].astype('category')

In [7]:
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col].fillna(df[col].mean(), inplace=True)
    if df[col].dtype == 'category':
        df[col].fillna(method='bfill', inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7729 entries, 0 to 7728
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   name                     7312 non-null   object        
 1   date                     7729 non-null   datetime64[ns]
 2   manner_of_death          7729 non-null   object        
 3   armed                    7520 non-null   object        
 4   age                      7247 non-null   float64       
 5   gender                   7709 non-null   object        
 6   race                     6269 non-null   object        
 7   city                     7729 non-null   object        
 8   state                    7729 non-null   object        
 9   signs_of_mental_illness  7729 non-null   bool          
 10  threat_level             7729 non-null   object        
 11  flee                     6854 non-null   object        
 12  body_camera              7729 non-

### <font color='green'>EDA</font>

In [9]:
df.head(2)

,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact,year,month,day_name
0,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True,2015,January,Friday
1,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True,2015,January,Friday


In [10]:
manner_death_df = df.groupby(['manner_of_death', 'armed']).size().reset_index(name='counts')
manner_death_df.head(2)

,manner_of_death,armed,counts
0,shot,Airsoft pistol,6
1,shot,BB gun,19


In [12]:
age_df = df.groupby(['state']).median()[['age']].reset_index()
age_df.head(2)

/tmp/ipykernel_6411/1328631959.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  age_df = df.groupby(['state']).median()[['age']].reset_index()


,state,age
0,AK,34.000000
1,AL,37.178971


In [14]:
yearMD_df = df.groupby(['year', 'month', 'day_name']).size().reset_index(name='count')
yearMD_df.head(2)

,year,month,day_name,count
0,2015,April,Friday,10
1,2015,April,Monday,6


In [15]:
month_df = df.groupby(['month', 'day_name']).size().reset_index(name='count')
month_df.head(2)

,month,day_name,count
0,April,Friday,83
1,April,Monday,72


In [42]:
city_df = df.groupby(['state', 'year']).size().reset_index(name='counts').sort_values(['counts'], ascending=False)
city_df['counts'].max()


190

In [43]:
city_df.head(2)

,state,year,counts
32,CA,2015,190
34,CA,2017,160


In [44]:
df[df['state'] == 'CA'].groupby(['year']).size().reset_index(name='counts')

,year,counts
0,2015,190
1,2016,139
2,2017,160
3,2018,116
4,2019,135
5,2020,147
6,2021,141
7,2022,86


In [18]:
yearMD_df = df.groupby(['month', 'year']).size().reset_index(name='count')
yearMD_df.head(2)

,month,year,count
0,1,2015,76
1,1,2016,81


In [19]:
yearMD_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   month   93 non-null     int64
 1   year    93 non-null     int64
 2   count   93 non-null     int64
dtypes: int64(3)
memory usage: 2.3 KB
